In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing as fch

import seaborn as sns 
from matplotlib import pyplot as plt
 
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_squared_error as mse

from sklearn.cluster import KMeans


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
california_housing = fch(as_frame=True).frame

In [ ]:
# Функции нормализации значений по минимаксу и для z-масштабирования (реализовал как функции, хотя можно использовать StandardScaler)

def normalize (dt, strg, dt_min, dt_max):
  return (dt[strg] - dt_min) / (dt_max - dt_min)
  
def z_scale(dt, strg, dt_mean, dt_stdev):
  return (dt[strg] - dt_mean) / dt_stdev

In [ ]:
# Дополнительные признаки, полученные в результате эффектов взаимодействия (новые признаки из имеющихся данных):

california_housing['Bedrooms_per_Room'] = california_housing['AveBedrms'] / california_housing['AveRooms']  # - количество спален на каждую комнату
california_housing['Total_Avg_Rooms'] = california_housing['AveRooms'] + california_housing['AveBedrms'] # - сумма всех комнат (спальни + другие комнаты)
california_housing['Population_per_Room'] = california_housing['Population'] / california_housing['Total_Avg_Rooms'] # - среднее кол-во человек на каждую комнату
california_housing['MedInc_norm'] = california_housing.apply(normalize, strg='MedInc', dt_min = california_housing['MedInc'].min(),  dt_max = california_housing['MedInc'].max(), axis=1)
km = KMeans(n_clusters = 25)
cluster = km.fit_predict(california_housing[['Latitude', 'Longitude']])
california_housing['Region_cluster'] = cluster 

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
housing_names = ['MedInc', 'HouseAge','AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude', 'Bedrooms_per_Room', 'Total_Avg_Rooms', 'Population_per_Room', 'MedInc_norm', 'Region_cluster'] # - список столбцов

In [ ]:
california_housing.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal,Bedrooms_per_Room,Total_Avg_Rooms,Population_per_Room,MedInc_norm,Region_cluster
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526,0.146591,8.007937,40.210109,0.539668,4
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585,0.155797,7.210018,333.008897,0.538027,4
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521,0.129516,9.361582,52.982498,0.466028,4
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413,0.184458,6.890411,80.982107,0.354699,4
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422,0.172096,7.362934,76.735711,0.230776,4


In [ ]:
X_data = california_housing[housing_names]
y_data = california_housing['MedHouseVal']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=20 )
housing_model = LinearRegression()
housing_model.fit(X_train, y_train)

predicted_data = housing_model.predict(X_test)
print('predicted_data: ', predicted_data)

predicted_data_train = housing_model.predict(X_train)
print('predicted_data_train: ', predicted_data_train)

print(housing_model.score(X_train, y_train))
print(housing_model.score(X_test, y_test))

mse_data = mse(y_test, predicted_data)
print(mse_data)
# print(np.sqrt(mse_data))

mse_data_train = mse(y_train, predicted_data_train)
print(mse_data_train)

predicted_data:  [1.57868605 2.99957163 1.77108504 ... 1.75578463 2.82586997 1.17070104]
predicted_data_train:  [1.74155561 1.89681747 2.34376916 ... 2.26669745 3.03745231 2.55776053]
0.6122979778273303
0.6231701081065317
0.5256547210671059
0.5100997138592676


In [ ]:
housing_decision_tree = DecisionTreeRegressor(random_state=20)
housing_decision_tree.fit(X_train, y_train)

housing_dt_predictions = housing_decision_tree.predict(X_test)
tree_mse = mse(y_test, housing_dt_predictions)
tree_rmse = np.sqrt(tree_mse)

train_score = housing_decision_tree.score(X_train, y_train)
test_score = housing_decision_tree.score(X_test, y_test)

In [ ]:
print('Предсказанные значения: ', housing_dt_predictions)
print('Средняя квадратическая ошибка: ', tree_mse)
print('Корень средней квадратической ошибки: ', tree_rmse)
print('Точность модели на тренировочной выборке: ', train_score)
print('Точность модели на тестовой выборке: ', test_score)

Предсказанные значения:  [1.293 2.636 1.555 ... 1.234 2.086 0.823]
Средняя квадратическая ошибка:  0.5193198605811288
Корень средней квадратической ошибки:  0.7206385089496181
Точность модели на тренировочной выборке:  1.0
Точность модели на тестовой выборке:  0.6277114252419412


Как видно из результатов выше, произошло переобучение модели - точность модели на тренировочных данных оказалась равной 100%, а на тестовых данных - 62,78%. В связи с чем необходимо перенастроить модель, используя дополнительные параметры

In [ ]:
def decisionTreeRegressorOptimizer(X_trn, X_tst, y_trn, y_tst, max_depth):
  for i in range(max_depth):
        model = DecisionTreeRegressor(random_state=20, max_depth=(i + 1))
        model.fit(X_trn, y_trn)

        housing_dt_predictions = model.predict(X_tst)
        tree_mse = mse(y_tst, housing_dt_predictions)
        tree_rmse = np.sqrt(tree_mse)

        train_score = model.score(X_trn, y_trn)
        test_score = model.score(X_tst, y_tst)

        print(f'max_depth: {i+1}')
        print('Предсказанные значения: ', housing_dt_predictions)
        print('Средняя квадратическая ошибка: ', tree_mse)
        print('Корень средней квадратической ошибки: ', tree_rmse)
        print('Точность модели на тренировочной выборке: ', train_score)
        print('Точность модели на тестовой выборке: ', test_score)
        print('----------------------------------------------------------')

def decisionTreeRegressorOptimizer_samples_leaf(X_trn, X_tst, y_trn, y_tst, min_samples_leaf):
  for i in range(min_samples_leaf):
        model = DecisionTreeRegressor(random_state=20, max_depth=9, min_samples_leaf=(i+1))
        model.fit(X_trn, y_trn)

        housing_dt_predictions = model.predict(X_tst)
        tree_mse = mse(y_tst, housing_dt_predictions)
        tree_rmse = np.sqrt(tree_mse)

        train_score = model.score(X_trn, y_trn)
        test_score = model.score(X_tst, y_tst)

        print(f'min_samples_leaf: {i+1}')
        print('Предсказанные значения: ', housing_dt_predictions)
        print('Средняя квадратическая ошибка: ', tree_mse)
        print('Корень средней квадратической ошибки: ', tree_rmse)
        print('Точность модели на тренировочной выборке: ', train_score)
        print('Точность модели на тестовой выборке: ', test_score)
        print('----------------------------------------------------------')

In [ ]:
decisionTreeRegressorOptimizer(X_train, X_test, y_train, y_test, 25)

max_depth: 1
Предсказанные значения:  [1.73858948 3.29137813 1.73858948 ... 1.73858948 3.29137813 1.73858948]
Средняя квадратическая ошибка:  0.942802843944017
Корень средней квадратической ошибки:  0.9709803519865976
Точность модели на тренировочной выборке:  0.3061759759814944
Точность модели на тестовой выборке:  0.32412612400959817
----------------------------------------------------------
max_depth: 2
Предсказанные значения:  [1.37617238 2.91163646 1.37617238 ... 2.10487101 2.91163646 1.37617238]
Средняя квадратическая ошибка:  0.750560031089045
Корень средней квадратической ошибки:  0.8663486775479288
Точность модели на тренировочной выборке:  0.4429923774953122
Точность модели на тестовой выборке:  0.4619406160746037
----------------------------------------------------------
max_depth: 3
Предсказанные значения:  [1.71612869 2.65104074 1.22315435 ... 1.90224783 2.65104074 1.71612869]
Средняя квадратическая ошибка:  0.6407556111093196
Корень средней квадратической ошибки:  0.80047

Наилучший компромисс между точностью на тренировочной и тестовой выборках определяется при значении параметра max_depth = 9 (78% и 70% соответственно)

In [ ]:
decisionTreeRegressorOptimizer_samples_leaf(X_train, X_test, y_train, y_test, 25)

min_samples_leaf: 1
Предсказанные значения:  [1.51017582 2.24627083 1.58473077 ... 2.26960451 2.32211621 0.93997247]
Средняя квадратическая ошибка:  0.4348846940442092
Корень средней квадратической ошибки:  0.6594578789006992
Точность модели на тренировочной выборке:  0.7880954522006867
Точность модели на тестовой выборке:  0.688241072181139
----------------------------------------------------------
min_samples_leaf: 2
Предсказанные значения:  [1.61745714 2.24627083 1.58473077 ... 2.26960451 2.32211621 0.93997247]
Средняя квадратическая ошибка:  0.4276333696281718
Корень средней квадратической ошибки:  0.6539368238814601
Точность модели на тренировочной выборке:  0.7886109596387327
Точность модели на тестовой выборке:  0.6934393814253378
----------------------------------------------------------
min_samples_leaf: 3
Предсказанные значения:  [1.61745714 2.27243182 1.58473077 ... 2.26960451 2.32211621 0.93997247]
Средняя квадратическая ошибка:  0.42717943736151615
Корень средней квадратич

Наилучший компромисс между точностью на тренировочной и тестовой выборках определяется при значении параметра min_samples_leaf = 10 (77,1% и 71,89% соответственно)

Сформулируйте выводы по проделанной работе.

a. Сравните точность двух моделей.

b. Напишите свое мнение, для каких задач предпочтительнее использовать обученные в работе модели? Какие у них есть плюсы и минусы?

Для получения зачета по этому домашнему заданию, должно быть как минимум реализовано обучение двух моделей, выведена их точность, оптимизирован один параметр дерева решений.

Выводы: 

а) Модель DecisionTreeRegressor показывает более высокую точность предсказания по сравнению с моделью линейной регрессии. Однако DecisionTreeRegressor склонна к переобучению, поэтому во избежание последнего необходимо подбирать оптимальные параметры модели

b) Модель на основе линейной регрессии отличается простотой и может быть использована в случаях, где наблюдается сильная корреляция / линейная взаимосвязь между признаками и целевой переменной; DecisionTreeRegressor является более сложной и ресурсозатратной моделью, но она может показать более высокую точность предсказания в случаях отсутствия явных корреляционных взаимоотношений между признаками и целевой переменной.